<a href="https://colab.research.google.com/github/JSassaki/library_assistant/blob/main/library_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assistente de biblioteca
**Autor:** Jorge Sassaki | [Linkedin](www.linkedin.com/in/jsassaki) | [Github](https://github.com/JSassaki)

O **objetivo** deste projeto é simples, **ajudar a recomendar livros para estudantes** baseado na entrada de um título de livro.

O assistente identifica o livro e retorna o nome do autor, só por sanity check. Caso o nome do autor esteja errado, pode ser que o modelo esteja alucinando ou que tenha confundido o livro!

Em seguida, dá a faixa etária mínima recomendada para o livro. Após isso, recomenda livros para diferentes faixas etárias que os estudantes podem ter. Isso serve para duas coisas:
1.   Caso o estudante seja mais jovem que o recomendado, provê alternativas adequadas à faixa etária do estudante;
2.   Caso o estudante tenha gostado de um livro e queira outros similares, provê alternativas também alternativas para ele.

Este projetinho foi inspirado pela necessidade real de uma amiga bibliotecária, que sempre fica na dúvida se alguma obra é adequada para a idade dos estudantes curiosos.

**To-dos:**
*   **Evitar que o modelo retorne o mesmo livro múltiplas vezes.** Por vezes, o modelo repete o livro em línguas diferentes, ou recomenda o mesmo livro para duas faixas etárias.
*   **Adicionar mais controles para a consulta.** No momento, a única entrada é o título do livro. Quero adicionar mais opções como gêneros, movimento literário, para filtrar melhor as recomendações.
*   **Melhorias de interface.**

**Problemas conhecidos:**
*    O retorno do gemini pode vir com "*finish_reason:* 5", sem resultado.
*    O modelo tem tendência a recomendar livros com uma temática similar, mas sem levar em conta os outros temas do livro.
   *   Por exemplo: Heartstopper, um livro sobre romance adolescente, que trata temas como família e identidade. Um dos livros recomendados foi Anne Frank, não está completamente errado, mas a "vibe" é completamente diferente.
*   Para idades menores, o modelo parece ter um viés em recomendar Ziraldo e O Pequeno Príncipe. São excelentes recomendações, mas fica repetitivo.
*   Por algum motivo, a integração com o gradio só parece funcionar se estiver em modo debug.

## Inicialização do ambiente

Instalação e importação dos pacotes necessários.

In [2]:
!pip install -q -U google-generativeai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.1 MB/s eta 

In [3]:
import pathlib
import textwrap

import google.generativeai as genai
import gradio as gr

from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata

# Não esqueça de autorizar o notebook a acessar sua chave
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


## Implementação:

Neste projeto, foi usado o few-shot prompt para exemplificar como o retorno deve ser formatado.

As idades dos estudantes foram delimitados para garantir que haveriam recomendações para todas as idades, mas sem ficar verboso demais. As faixas foram estipuladas por uma bibliotecária.

Movimentos literários como Naturalismo podem não ser recomendados para estudantes muito jovens, por isso também é pedido temas semelhantes.

Foi necessário instruir o modelo a responder que desconhece o livro, para evitar alucinações, ou evitar respostas que não eram recomendações. Por exemplo, para o livro "O Professor de Português Enlouqueceu de Vez", o assistente dava dicas de como se comunicar com o professor, pois não conhecia o livro e achava ser uma prompt comum.

Foi necessário limitar as travas do modelo para apenas "block few", pois livros naturalistas ou que tratam de racismo eram frequentemente bloqueados.

In [4]:
system_instruction="""Você auxilia um bibliotecário de uma escola a recomendar livros para os estudantes.
Dado o nome de um livro, identifique o autor e indique a idade recomendada para sua leitura. Justifique a idade recomendada.
Depois, aponte alternativas do mesmo movimento literária considerando as faixas etárias de 6-8; 9-11; 12-14 e 15-17.
Caso o movimento não seja adequado para alguma faixa etária, recomende livros que abordam temas similares.
Caso não conheça livro com o nome dado, diga em uma linha que não conhece o livro.
Exemplos:
---
## O Senhor dos Anéis

**Autor:** J.R.R. Tolkien

**Idade recomendada:** 12-14 anos

**Justificativa:** "O Senhor dos Anéis" é uma obra de alta fantasia com universo complexo, mitologia própria e linguagem elaborada em diversos momentos. A narrativa épica, com longas descrições e batalhas, exige certa maturidade e capacidade de acompanhar tramas paralelas e desenvolvimento de personagens. Apesar de conter elementos que atraem leitores mais jovens, como aventura e magia, a densidade da obra a torna mais adequada para leitores a partir de 12 anos, que já possuem maior capacidade de concentração e compreensão de temas como amizade, lealdade, corrupção e poder.

**Alternativas por faixa etária:**

**6-8 anos:** Obras com fantasia, aventura e linguagem simples são ideais.

* **Onde Vivem os Monstros (Maurice Sendak):** Uma criança explora seus sentimentos através da fantasia, encontrando monstros que representam seus medos e frustrações.
* **A História Interminável (Michael Ende):** Um garoto se refugia em um livro mágico e precisa salvar o mundo da fantasia.

**9-11 anos:** Aventuras com personagens cativantes e temas como amizade e coragem, introduzindo elementos de fantasia mais elaborados.

* **As Crônicas de Nárnia (C.S. Lewis):** Irmãos descobrem um mundo mágico dentro de um guarda-roupa e vivem grandes aventuras.
* **Percy Jackson e o Ladrão de Raios (Rick Riordan):** Um garoto descobre ser filho de um deus grego e embarca em uma jornada para salvar o Olimpo.

**12-14 anos:**  Fantasia com universos complexos, personagens bem desenvolvidos e temas mais maduros, como o amadurecimento, a perda e o heroísmo.

* **Eragon (Christopher Paolini):** Um jovem encontra um ovo de dragão e se torna um Cavaleiro de Dragão, lutando contra um rei tirano.
* **A Bússola de Ouro (Philip Pullman):** Uma garota viaja por universos paralelos em uma aventura que questiona o poder e a liberdade.

**15-17 anos:** Obras de fantasia épica com temas complexos, personagens densos e linguagem elaborada, explorando questões políticas, sociais e filosóficas.

* **Crônicas Saxônicas (Bernard Cornwell):** Jovem saxão criado por vikings precisa escolher um lado durante a invasão viking na Inglaterra.
* **O Nome do Vento (Patrick Rothfuss):** Um herói lendário narra sua vida, cheia de magia, aventura e tragédia.

---
## O Cortiço
**Autor:** Aluísio Azevedo

**Idade recomendada:** 15-17 anos

**Justificativa:** "O Cortiço" é uma obra naturalista que retrata a realidade crua e brutal da vida no Rio de Janeiro do final do século XIX. A obra aborda temas como pobreza, violência, exploração, alcoolismo, adultério e racismo de forma explícita e, por vezes, chocante. A linguagem utilizada também apresenta termos chulos e descrições naturalistas do corpo e de seus processos, o que pode ser inadequado para leitores mais jovens. A complexidade social retratada e a crítica social implícita na obra exigem maturidade e capacidade de análise por parte do leitor, tornando-a mais adequada para estudantes a partir de 15 anos.

**Alternativas por faixa etária:**

**6-8 anos:** Dada a temática complexa e a linguagem utilizada em "O Cortiço", não é recomendado apresentar obras similares para essa faixa etária. Em vez disso, podemos oferecer livros que explorem a vida em comunidade e a importância da amizade:

* **O Menino que Morava no Fim da Rua (Henriqueta Collins & Satoshi Kitamura):** Um menino tímido cria um mundo de fantasia para lidar com a solidão e faz um amigo especial.
* **A Família Biscoito (Isabella Fiorentino):** Uma família de biscoitos demonstra a importância da união, do respeito e da amizade.
**9-11 anos:** Ainda é cedo para abordar temas como os explorados em "O Cortiço" com essa faixa etária. Podemos apresentar livros que trabalhem a diversidade social e a empatia:

* **Extraordinário (R.J. Palacio):** Um garoto com deformidade facial luta para ser aceito em sua escola e nos ensina sobre compaixão e amizade.
* **A Menina que Roubava Livros (Markus Zusak):** Ambientada na Alemanha nazista, acompanhamos a história de uma menina que encontra conforto nos livros e na amizade em meio à guerra.
**12-14 anos:** Podemos começar a introduzir obras que explorem a desigualdade social e a realidade brasileira, mas com abordagem mais leve e personagens cativantes:

* **Capitães da Areia (Jorge Amado):** Um grupo de meninos de rua luta pela sobrevivência em Salvador, revelando a pobreza e a violência que enfrentam.
* **Vidas Secas (Graciliano Ramos):** Uma família de retirantes enfrenta a seca e a miséria no sertão nordestino, retratando a realidade social da época.
**15-17 anos:** É possível apresentar obras que explorem o Naturalismo e retratem a realidade social brasileira de forma mais crua e crítica:

* **Dom Casmurro (Machado de Assis):** Um clássico da literatura brasileira que levanta questões sobre ciúmes, traição e a complexidade das relações humanas.
* **O Mulato (Aluísio Azevedo):** Uma obra que denuncia o racismo e a hipocrisia da sociedade maranhense do século XIX.

---
## Crepúsculo
**Autor:** Stephenie Meyer

**Idade recomendada:** 12-14 anos

**Justificativa:** "Crepúsculo" é uma obra de fantasia romântica que aborda temas como amor, imortalidade e a busca pela identidade. A linguagem é acessível e a narrativa focada no romance atrai principalmente jovens leitores. Apesar de conter elementos sobrenaturais, a trama gira em torno de conflitos adolescentes e a descoberta do primeiro amor, tornando-a adequada para leitores a partir de 12 anos, que já se identificam com tais temas.

**Alternativas por faixa etária:**

**6-8 anos:** Fantasia com foco em amizade e aventura, com linguagem simples e temas adequados à faixa etária.

    * **O Pequeno Príncipe (Antoine de Saint-Exupéry):** Um piloto perdido no deserto encontra um príncipe de outro planeta, aprendendo sobre amor, amizade e a importância de se importar com os outros.
    * **A Menina e o Unicórnio (Peter S. Beagle):** Uma garota solitária encontra um unicórnio e embarca em uma jornada mágica em busca de outros de sua espécie.
**9-11 anos:** Aventuras com protagonistas jovens, temas como amizade e coragem, introduzindo elementos de romance e fantasia.

* **Matilda (Roald Dahl):** Uma menina com inteligência excepcional enfrenta desafios em casa e na escola, usando seus poderes para se defender e proteger seus amigos.
* **Ponte para Terabítia (Katherine Paterson):** Dois amigos criam um mundo mágico na floresta para lidar com os desafios da vida real, explorando temas como amizade, perda e imaginação.
**12-14 anos:** Obras de fantasia com protagonistas adolescentes, explorando temas como amor, identidade e relacionamentos, com elementos de suspense e mistério.

* **Diários de um Vampiro (L.J. Smith):** Uma adolescente se apaixona por um vampiro, vivenciando um triângulo amoroso e descobrindo segredos sobrenaturais.
* **A Seleção (Kiera Cass):** Em um futuro distópico, 35 garotas competem pela mão do príncipe, explorando temas como amor, poder e desigualdade social.
**15-17 anos:** Obras de fantasia com temas mais complexos, como a natureza humana, o livre arbítrio e a moralidade, com narrativas envolventes e personagens densos.

* **O Morro dos Ventos Uivantes (Emily Brontë):** Uma história de amor obsessivo e vingança ambientada nas charnecas da Inglaterra, explorando temas como paixão, isolamento e tragédia.
* **A Princesa Prometida (William Goldman):** Uma história clássica de amor, aventura e humor, que satiriza os contos de fadas e apresenta personagens memoráveis.
"""
safety_settings={
    "HARASSMENT": "BLOCK_ONLY_HIGH",
    "HATE": "BLOCK_ONLY_HIGH",
    "SEXUAL": "BLOCK_ONLY_HIGH",
    "DANGEROUS": "BLOCK_ONLY_HIGH"
}
model = genai.GenerativeModel('gemini-1.5-pro-latest',
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

In [7]:
def ask_gemini(Livro):
    """
    Simplesmente faz a consulta com o gemini e retorna o output do modelo ou um erro.
    Inputs:
      Livro: String do nome do livro consultado.

    Outputs: String no formato markdown.
    """
    try:
        response=model.generate_content(Livro)
        return response.text
    except:
        # Caso o retorno venha vazio ou haja algum problema na conexão com o modelo
        return "Ocorreu um erro, tente novamente."

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("""# Assistente de biblioteca
    Insira o nome do livro a ser consultado, de preferência completo.

    O Assistente identifica o autor para confirmar se o livro está correto, e dá idade recomendada para o livro com uma justificativa.

    Caso o aluno seja jovem demais, ou esteja procurando livros similares para sua idade, também retorna recomendações para faixas etárias dos 6 aos 17 anos.

    Este assistente responde com o uso de IA e podem ocorrer erros. É recomendado o uso da experiência e bom-senso do usuário ao repassar as recomendações aos estudantes.

    """)
    book = gr.Textbox(label="Nome do livro")
    rec_btn = gr.Button("Recomendar")
    output = gr.Markdown(label="Recomendação")
    rec_btn.click(fn=ask_gemini, inputs=book, outputs=output, api_name="Recomendar")

demo.launch(debug=True) # por algum motivo só funciona em modo debug

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://db6b6d4f41a8564ef8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
